In [1]:
import os
import csv
import json

In [2]:
with open(
    "/Users/rachelbittner/mir_datasets/Orchset/Orchset - Predominant Melodic Instruments.csv",
    'r') as fhandle:
    reader = csv.reader(fhandle, delimiter=',')
    raw_data = []
    for line in reader:
        if line[0] == 'excerpt':
            header = line
            continue
        raw_data.append(line)


In [3]:
header

['excerpt',
 'Predominant melodic instrument(s) description',
 'Alternating',
 'containsstrings',
 'containswinds',
 'containsbrass',
 'Onlystrings',
 'Onlywinds',
 'Onlybrass']

In [4]:
TF = {'TRUE': True, 'FALSE': False}

index = {}
for line in raw_data:
    track_id = line[0].split('.')[0]

    id_split = track_id.split('.')[0].split('-')
    if id_split[0] == 'Musorgski' or id_split[0] == 'Rimski':
        id_split[0] = '-'.join(id_split[:2])
        id_split.pop(1)
    
    melodic_instruments = [s.split(',') for s in line[1].split('+')]
    melodic_instruments = [item.lower() for sublist in melodic_instruments for item in sublist]
    for i, inst in enumerate(melodic_instruments):
        if inst == 'string':
            melodic_instruments[i] = 'strings'
        elif inst == 'winds (solo)':
            melodic_instruments[i] = 'winds'
    melodic_instruments = list(set(melodic_instruments))

    index[track_id] = {
        'predominant_melodic_instruments-raw': line[1],
        'predominant_melodic_instruments-normalized': melodic_instruments,
        'alternating_melody': TF[line[2]],
        'contains_winds': TF[line[3]],
        'contains_strings': TF[line[4]],
        'contains_brass': TF[line[5]],
        'only_strings': TF[line[6]],
        'only_winds': TF[line[7]],
        'only_brass': TF[line[8]],
        'composer': id_split[0],
        'work': '-'.join(id_split[1:-1]),
        'excerpt': id_split[-1][2:],
        'audio_path_stereo': 'Orchset/audio/stereo/{}.wav'.format(track_id),
        'audio_path_mono': 'Orchset/audio/mono/{}.wav'.format(track_id),
        'melody_path': 'Orchset/GT/{}.mel'.format(track_id)
    }

In [7]:
with open("../mir_dataset_loaders/indexes/orchset_index.json", 'w') as fhandle:
    json.dump(index, fhandle, indent=2)